<a href="https://colab.research.google.com/github/Yuan-Chun-Chih/114-1-internet/blob/main/HW_1_V2_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#這是一個在 Google Colab 上運行的 Python 程式，用於管理與分析日常支出。
#程式會自動從指定的 Google 試算表 (Google Sheets) 中讀取「消費紀錄」分頁，計算總支出、分類小計、AA 平均分攤、付款人淨額以及清算建議，並將結果寫回到 Summary 分頁。同時，它還會針對「本週」的花費模式，給出簡單的理財建議。
#使用gradio進行分析

In [7]:
from google.colab import auth
auth.authenticate_user()


In [8]:
!pip -q install gradio

In [9]:
import gspread
from google.auth import default
import pandas as pd
from datetime import datetime, timedelta
import pytz
import gradio as gr

# ====== 你的核心邏輯（維持最小改動） ======
def _open_sheet(sheet_url):
    creds, _ = default()
    gc = gspread.authorize(creds)
    sh = gc.open_by_url(sheet_url)
    return sh

def _read_table(sh, data_sheet_name):
    try:
        ws_data = sh.worksheet(data_sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        ws_data = sh.get_worksheet(0)
    rows = ws_data.get_all_values()
    if not rows:
        raise ValueError("資料表是空的。")
    headers = [c.strip().lower() for c in rows[0]]
    df = pd.DataFrame(rows[1:], columns=headers)
    expected = ["date", "category", "item", "amount", "payer"]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"缺少欄位：{missing}；請確認表頭為 {expected}")
    # 轉型
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce").fillna(0.0)
    df["category"] = df["category"].astype(str).str.strip()
    df["item"] = df["item"].astype(str).str.strip()
    df["payer"] = df["payer"].astype(str).str.strip()
    df = df.dropna(subset=["date"])
    df = df[df["amount"] > 0]
    return ws_data, df

def _settlement_plan(net_series):
    creditors = [(p, amt) for p, amt in net_series.items() if amt > 1e-8]
    debtors   = [(p, -amt) for p, amt in net_series.items() if amt < -1e-8]
    creditors.sort(key=lambda x: x[1], reverse=True)
    debtors.sort(key=lambda x: x[1], reverse=True)
    plans = []; i=j=0
    while i < len(debtors) and j < len(creditors):
        d_name, d_amt = debtors[i]; c_name, c_amt = creditors[j]
        pay = min(d_amt, c_amt)
        plans.append({"from": d_name, "to": c_name, "amount": round(float(pay), 2)})
        d_amt -= pay; c_amt -= pay
        if d_amt <= 1e-8: i += 1
        else: debtors[i] = (d_name, d_amt)
        if c_amt <= 1e-8: j += 1
        else: creditors[j] = (c_name, c_amt)
    return pd.DataFrame(plans)

def _weekly_window(tz_name="Asia/Taipei"):
    tz_taipei = pytz.timezone(tz_name)
    now = datetime.now(tz_taipei)
    weekday = now.weekday()                 # Monday=0
    week_start = (now - timedelta(days=weekday)).replace(hour=0, minute=0, second=0, microsecond=0)
    week_end   = week_start + timedelta(days=6, hours=23, minutes=59, seconds=59)
    # 轉為 tz-naive，與 df['date'] 一致
    return pd.Timestamp(week_start.replace(tzinfo=None)), pd.Timestamp(week_end.replace(tzinfo=None)), now

def _write_summary(sh, summary_name, meta, by_category, payer_out, settle_df, tips):
    try:
        ws_sum = sh.worksheet(summary_name)
    except gspread.exceptions.WorksheetNotFound:
        ws_sum = sh.add_worksheet(title=summary_name, rows=200, cols=20)
    ws_sum.clear()

    # 基本摘要
    ws_sum.update("A1", [["項目","內容"]])
    ws_sum.update(f"A2:B{1+len(meta)+1}", meta)

    # 分類小計（A10 起）
    start_row_cat = 10
    ws_sum.update(f"A{start_row_cat}", [["分類","小計"]])
    if not by_category.empty:
        ws_sum.update(f"A{start_row_cat+1}:B{start_row_cat+len(by_category)}", by_category.values.tolist())

    # 付款人小計 + 淨額（D10 起）
    ws_sum.update(f"D{start_row_cat}", [payer_out.columns.tolist()])
    if not payer_out.empty:
        ws_sum.update(f"D{start_row_cat+1}:F{start_row_cat+len(payer_out)}", payer_out.values.tolist())

    # 清算建議
    start_row_settle = start_row_cat + max(len(by_category), len(payer_out)) + 4
    ws_sum.update(f"A{start_row_settle}", [["清算建議（from → to, amount）"]])
    if not settle_df.empty:
        ws_sum.update(f"A{start_row_settle+1}", [["付款人(付出)","收款人(收款)","金額"]])
        ws_sum.update(
            f"A{start_row_settle+2}:C{start_row_settle+1+len(settle_df)}",
            settle_df.rename(columns={"from":"付款人(付出)","to":"收款人(收款)","amount":"金額"}).values.tolist()
        )
    else:
        ws_sum.update(f"A{start_row_settle+1}", [["所有人已平衡，無需清算"]])

    # 本週建議
    tip_row = start_row_settle + (len(settle_df) + 3 if not settle_df.empty else 3)
    ws_sum.update(f"A{tip_row}", [["本週花錢習慣與建議"]])
    if tips:
        ws_sum.update(f"A{tip_row+1}:A{tip_row+len(tips)}", [[t] for t in tips])

def run(sheet_url, data_sheet_name, summary_sheet_name, tz_name):
    # 開啟與讀取
    sh = _open_sheet(sheet_url)
    ws_data, df = _read_table(sh, data_sheet_name)

    # 基本統計
    total_spent = float(df["amount"].sum())
    by_category = df.groupby("category", dropna=False)["amount"].sum().sort_values(ascending=False).reset_index()
    by_payer = df.groupby("payer", dropna=False)["amount"].sum().sort_values(ascending=False).reset_index()

    # AA 與淨額
    payers = sorted(df["payer"].dropna().unique().tolist())
    n = max(len(payers), 1)
    equal_share = total_spent / n
    net = by_payer.set_index("payer")["amount"] - equal_share
    net = net.reindex(payers).fillna(-equal_share).round(2)
    settle_df = _settlement_plan(net)

    # 本週分析
    week_start_naive, week_end_naive, now = _weekly_window(tz_name)
    df_week = df[(df["date"] >= week_start_naive) & (df["date"] <= week_end_naive)]
    tips = []
    if df_week.empty:
        tips = ["本週尚無消費紀錄。"]
    else:
        cat_sum = df_week.groupby("category")["amount"].sum().sort_values(ascending=False)
        total = cat_sum.sum()
        if total > 0:
            top_cat = cat_sum.index[0]
            ratio = float(cat_sum.iloc[0] / total)
            if ratio > 0.4:
                tips.append(f"「{top_cat}」占本週總支出 {ratio:.0%}，支出集中，建議檢視是否可壓低。")
            day_sum = df_week.groupby(df_week["date"].dt.date)["amount"].sum().sort_values(ascending=False)
            if float(day_sum.iloc[0] / total) > 0.5:
                tips.append("單日支出占比偏高，留意集中消費的風險。")
        if not tips:
            tips = ["本週支出分布尚稱均衡，非必要支出可設定上限持續觀察。"]

    # 準備寫回 Summary
    meta = [
        ["Expense Summary (自動產生)", ""],
        ["產生時間", now.strftime("%Y-%m-%d %H:%M:%S %Z")],
        ["資料分頁", ws_data.title],
        ["期間（本週）", f"{week_start_naive.strftime('%Y-%m-%d')} ~ {week_end_naive.strftime('%Y-%m-%d')}"],
        ["總支出 (All)", total_spent],
        ["人數 (AA)", n],
        ["每人應分擔 (AA)", round(equal_share, 2)],
        ["", ""],
    ]

    # 組出付款人表（含淨額）做為 UI 預覽
    payer_table = by_payer.set_index("payer").copy()
    payer_table["淨額(>0代表代墊,<0代表應付)"] = net
    payer_out = payer_table.reset_index().rename(columns={"payer":"付款人","amount":"小計"})

    # 寫回 Google Sheet
    _write_summary(sh, summary_sheet_name, meta, by_category, payer_out, settle_df, tips)

    # 回傳給 UI 預覽
    msg = "✅ 已完成計算並寫入 Summary 分頁。"
    return (
        msg,
        by_category,         # DataFrame
        payer_out,           # DataFrame
        settle_df,           # DataFrame
        "\n".join(tips)      # 建議文字
    )

# ====== Gradio 介面 ======
with gr.Blocks(fill_height=True) as demo:
    gr.Markdown("## 日常支出速算與分攤（Google Sheets）")
    with gr.Row():
        sheet_url = gr.Textbox(label="Google Sheet 連結", placeholder="https://docs.google.com/spreadsheets/d/1Fv4pzfY7tP_WQImaCdycaJO2XLA5tZ3MJyLfBgropQE/edit?gid=0#gid=0")
    with gr.Row():
        data_sheet_name = gr.Textbox(label="資料分頁名稱", value="消費紀錄")
        summary_sheet_name = gr.Textbox(label="輸出分頁名稱", value="Summary")
        tz_name = gr.Dropdown(label="時區", choices=["Asia/Taipei","UTC"], value="Asia/Taipei")
    run_btn = gr.Button("執行")
    msg_out = gr.Textbox(label="狀態", interactive=False)
    cat_df = gr.Dataframe(label="分類小計", interactive=False)
    payer_df = gr.Dataframe(label="付款人小計與淨額", interactive=False)
    settle_df_out = gr.Dataframe(label="清算建議", interactive=False)
    tips_out = gr.Textbox(label="本週花錢習慣與建議", lines=6, interactive=False)

    run_btn.click(
        fn=run,
        inputs=[sheet_url, data_sheet_name, summary_sheet_name, tz_name],
        outputs=[msg_out, cat_df, payer_df, settle_df_out, tips_out],
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be587b80f33c4ff7ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
